In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [3]:
# from ml.preprocess import data
import pickle
from dask_ml.preprocessing import StandardScaler

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

import numpy as np
import pandas as pd

from itertools import combinations

from utils import helper
# from ml.models.base import brisk_nn_pytorch, slug_xgboost, slug_svm, slug_sdg
from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_xgboost import SlugXGBoost
from sklearn.model_selection import train_test_split

import xgboost as xgb


In [4]:
df_X, df_y = helper.get_covid_dataset()
df_X = df_X.drop(['location'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33)


X_scalar = StandardScaler().fit_transform(df_X)

X_base_scalar, X_test_scalar, y_base, y_test = train_test_split(X_scalar, df_y, test_size=0.33)
X_train_scalar, X_val_scalar, y_train, y_val = train_test_split(X_base_scalar, y_base)

In [5]:
slug_ann = SlugANN()

slug_ann.X_train = X_train_scalar
slug_ann.X_test = X_test_scalar
slug_ann.X_val = X_val_scalar
slug_ann.y_train = y_train
slug_ann.y_test = y_test
slug_ann.y_val = y_val



In [6]:
slug_ann.get_model_scores()

total models trained: 2


In [8]:
slug_ann.final_models[0]

[Sequential(
   (0): Linear(in_features=40, out_features=56, bias=True)
   (1): ReLU()
   (2): Linear(in_features=56, out_features=212, bias=True)
   (3): ReLU()
   (4): Linear(in_features=212, out_features=204, bias=True)
   (5): ReLU()
   (6): Linear(in_features=204, out_features=132, bias=True)
   (7): ReLU()
   (8): Linear(in_features=132, out_features=72, bias=True)
   (9): ReLU()
   (10): Linear(in_features=72, out_features=184, bias=True)
   (11): ReLU()
   (12): Linear(in_features=184, out_features=116, bias=True)
   (13): ReLU()
   (14): Linear(in_features=116, out_features=1, bias=True)
   (15): ReLU()
 ),
 -0.08001711067605166]

In [10]:

slug_xgboost = SlugXGBoost()
slug_xgboost.X_train = X_train
slug_xgboost.X_test = X_test
slug_xgboost.y_train = y_train
slug_xgboost.y_test = y_test


In [11]:
slug_xgboost.get_model_scores()

total models trained: 28


ValueError: feature_names mismatch: ['location', 'new_cases_per_million', 'population_cov', 'life_expectancy_cov', 'Population', 'Area_km2', 'Density_km2', 'Year_x', 'Meningitis', 'Neoplasms', 'Fire, heat, and hot substances', 'Malaria', 'Drowning', 'Interpersonal violence', 'HIV/AIDS', 'Drug use disorders', 'Tuberculosis', 'Road injuries', 'Maternal disorders', 'Lower respiratory infections', 'Neonatal disorders', 'Alcohol use disorders', 'Exposure to forces of nature', 'Diarrheal diseases', 'Environmental heat and cold exposure', 'Nutritional deficiencies', 'Self-harm', 'Conflict and terrorism', 'Diabetes mellitus', 'Poisonings', 'Protein-energy malnutrition', 'Cardiovascular diseases', 'Chronic kidney disease', 'Chronic respiratory diseases', 'Cirrhosis and other chronic liver diseases', 'Digestive diseases', 'Acute hepatitis', "Alzheimer's disease and other dementias", "Parkinson's disease", 'Total', 'air_polution'] ['new_cases_per_million', 'population_cov', 'life_expectancy_cov',

In [7]:
slug_ann.load_saved_models()

total models trained: 1


In [4]:
# xgb_model = brisk_xgboost.fetch_model()
# xgb_model.save_model(brisk_xgboost.MODEL_SAVE_PATH + 'xgboost.json')

In [5]:
# xgb_model = slug_xgboost.fetch_model()

In [6]:
# nn_model = brisk_nn_pytorch.fetch_model()

In [7]:
# nn_model.eval()

In [8]:
import os
import glob

import xgboost as xgb

from utils.helper import adjusted_r2


problem_type = 'regression'

if problem_type == 'regression':
    eval_metrics = [mean_squared_error, r2_score]
    min_max = min
else:
    eval_metrics = [f1_score, roc_auc_score, precision_score, recall_score]
    min_max = max


In [9]:

best_models = []

slug_xgboost_files = glob.glob(slug_xgboost.MODEL_SAVE_PATH+'/*')

for file in slug_xgboost_files:
    model = slug_xgboost.load_model(file)
    best_models.append(model)


In [10]:
len(best_models)

9

In [11]:
# type(model)
type(best_models[0])

xgboost.core.Booster

In [12]:


if xgb.core.Booster == type(best_models[0]):
    print('yes')

yes


In [13]:
### Predictions from NN Torch

X_train, X_test_scalar, y_train, y_test = brisk_nn_pytorch.get_split_dataset()
#X_test_scalar = StandardScaler().fit_transform(X_test)     
X_test_tensor = helper.df_to_tensor(X_test_scalar)


brisk_nn_pytorch_files = glob.glob(brisk_nn_pytorch.PARAM_MODEL_SAVE_PATH+'/*')


nn_models = []
r2_scores_nn_models = []
preds_nn_models = []
good_nn_models = []
good_nn_models_pred = []

for file in brisk_nn_pytorch_files:
    #xgb_best_model = xgb.Booster()
    # xgb_best_model.load_model(file)
    model = brisk_nn_pytorch.load_model(file)
    nn_models.append(model)



for model in nn_models:

    pred = model(X_test_tensor)
    pred = helper.torch_tensor_to_numpy(pred)
    pred = pred.reshape(pred.shape[0],)

    # comb_preds.append(pred)
    
    # for metric in eval_metrics[0]:
    metirc_score = r2_score(pred, y_test)

    if metirc_score> 0.4:
        good_nn_models.append(model)
        good_nn_models_pred.append(pred)


    r2_scores_nn_models.append(metirc_score)


r2_scores_nn_models


[0.2398409925115903,
 -6.7978184894659,
 -0.13156062523985912,
 -0.8702828447903668,
 -0.32586625555931725,
 0.26507808319570314,
 -0.34497131097944966,
 0.4104746991798226,
 -0.048707386213444614,
 -0.8461573299484177,
 0.26145439687207983,
 0.43644399603736106,
 0.19935548014418647]

In [14]:
### Predictions from XGBOSST

X_train_orig, X_test_orig, _, _ = slug_xgboost.get_split_dataset([])
X_train, X_test, y_train, y_test = slug_xgboost.get_split_dataset(drop_list=['location'])
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


xgb_models = []
r2_scores_xgb_models = []
preds_xgb_models = []
good_xgb_models = []
good_xgb_models_pred = []


slug_xgboost_files = glob.glob(slug_xgboost.MODEL_SAVE_PATH+'/*')


# print(brisk_nn_pytorch_files)

for file in slug_xgboost_files:
    #xgb_best_model = xgb.Booster()
    # xgb_best_model.load_model(file)
    model = slug_xgboost.load_model(file)
    xgb_models.append(model)



for model in xgb_models:
    pred = model.predict(dtest)
    preds_xgb_models.append(pred)
    
    # for metric in eval_metrics[0]:
    metirc_score = r2_score(pred, y_test)

    if metirc_score> 0.4:
        good_nn_models.append(model)
        good_xgb_models_pred.append(pred)

    r2_scores_xgb_models.append(metirc_score)


r2_scores_xgb_models



[0.7016786597577798,
 0.6036870860535384,
 0.7038825452392524,
 0.6916054254949667,
 0.6966467751943459,
 0.6933923857927424,
 0.6018687572522508,
 0.7502451501608368,
 0.6915931273577398]

In [15]:
good_xgb_models_pred.extend(good_nn_models_pred)
good_pred = good_xgb_models_pred

In [16]:
# good_pred

In [17]:
df_good_pred = pd.DataFrame(good_pred).T
# df_good_pred['target'] = y_test.values
df_good_pred.shape
X_train, X_test, y_train, y_test = slug_xgboost.get_split_dataset(drop_list=['location'])


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_good_pred, y_test, test_size=0.33)


TypeError: Singleton array array(0.33) cannot be considered a valid collection.

In [20]:
# slug_sdg.fetch_model(df_good_pred, y_test)

In [19]:
# slug_svm.Params.pred_type='regression'
# slug_svm.fetch_model(df_good_pred, y_test)

In [171]:
#### Now fit SVM on the final predictions
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

df_good_pred_scalar = StandardScaler().fit_transform(df_good_pred)     
y_test_1d = y_test.values.reshape(y_test.shape[0])

regr_scr = SVR(C=1.0, epsilon=0.2)

regr_scr.fit(df_good_pred_scalar, y_test_1d)

SVR(epsilon=0.2)

In [ ]:
pred_type = "regression"

In [174]:
pred_scr = regr_scr.predict(df_good_pred_scalar)
r2_score(pred_scr, y_test_1d)


-9401.052727981196

In [157]:
df_good_pred

,0,1,2,3,4,5,6,7,8,9,10,target
0,2282.944824,2390.147705,2300.608887,2307.854004,2338.407715,2294.193848,2383.937500,2368.414795,2307.442627,926.766724,1671.106201,2342.551
1,1175.548828,1107.300415,1047.958130,1058.755981,1047.499023,1050.636475,1105.395996,1043.604736,1058.906494,524.894958,282.477142,1044.585
2,780.710510,1248.273438,1238.329590,1236.956299,1258.007324,1234.862427,1246.922363,1692.000244,1237.315552,1501.027222,1336.665161,2168.010
3,74.397934,59.714867,42.514999,42.377308,39.752472,41.601376,59.784386,31.256008,42.314976,81.320938,92.755135,55.977
4,24.961271,48.324459,44.217270,42.532051,45.624771,41.832939,48.331623,13.971548,42.502312,54.103924,69.669968,14.843
...,...,...,...,...,...,...,...,...,...,...,...,...
59,1134.570557,1197.655273,1260.438599,1249.295776,1281.590454,1243.509399,1196.239746,1210.959473,1249.630005,882.783691,1417.531738,1297.107
60,968.090576,556.462097,649.882080,666.639709,620.892578,657.279358,556.071350,861.049500,666.892456,612.543945,667.631470,981.817
61,70.886826,91.033760,28.431480,29.501001,25.176630,29.008516,90.970711,0.600573,29.495853,1545.730957,861.057434,0.000
62,1005.622986,1451.975342,1075.823486,1067.328613,1118.445190,1057.408447,1451.124634,1288.814453,1067.477783,797.647034,776.435669,2465.522


In [230]:
type(y_test)

pandas.core.frame.DataFrame

In [216]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_good_pred, y_test, test_size=0.25)


ValueError: Found input variables with inconsistent numbers of samples: [64, 16]

In [224]:
X_train.shape, X_test.shape


((48, 11), (16, 11))

In [227]:
# train_test_split(y_train, y_train, test_size=0.25)
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=3)



for train_index, test_index in kf.split(X_train, y_train):
    temp_train_x, temp_test_x = X_train.iloc[train_index], X_train.iloc[test_index]




In [204]:
# X_train
# y_train
# y_test

,y
109,2342.551
74,1044.585
162,2168.010
96,55.977
124,14.843
...,...
86,1297.107
131,981.817
40,0.000
173,2465.522


In [240]:
# df_good_pred_scalar
df_good_pred.shape, y_test.shape

((64, 11), (64, 1))

In [244]:
y_test.iloc[62], df_good_pred.iloc[62]

(y    2465.522
 Name: 173, dtype: float64,
 0     1005.622986
 1     1451.975342
 2     1075.823486
 3     1067.328613
 4     1118.445190
 5     1057.408447
 6     1451.124634
 7     1288.814453
 8     1067.477783
 9      797.647034
 10     776.435669
 Name: 62, dtype: float32)

In [253]:
slug_svm.fetch_model(df_good_pred, y_test)

Dask dashboard is available at http://127.0.0.1:8787/status
########## Re-running the discovery process ###############
Rerun #: 0
Starting train for trials:300


[W 2022-10-10 21:35:28,853] Trial 6 failed because of the following error: KeyError('[23, 25, 27, 28, 29, 31, 32, 34, 35, 36, 38, 39, 41, 42, 43, 46, 47, 48, 49, 50, 52, 53, 57, 58, 59, 62] not in index')
Traceback (most recent call last):
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/optuna/_optimize.py", line 189, in _run_trial
    value = func(trial)
  File "/mnt/c/Users/rwmas/GitHub/auto-learn/notebooks/../ml/models/base/slug_svm.py", line 124, in <lambda>
    func = lambda trial: objective(trial, df_X, df_y)
  File "/mnt/c/Users/rwmas/GitHub/auto-learn/notebooks/../ml/models/base/slug_svm.py", line 79, in objective
    for train_index, test_index in kf.split(df_X, df_y):
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/pandas/core/indexing.py", line 967, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/pandas/core/indexing.py", line 119

KeyError: '[23, 25, 27, 28, 29, 31, 32, 34, 35, 36, 38, 39, 41, 42, 43, 46, 47, 48, 49, 50, 52, 53, 57, 58, 59, 62] not in index'

In [52]:
comb_lens = np.arange(1, len(best_models))
comb_lens

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21])

In [ ]:
            res = dask.delayed(simmulate)(param, i)
            lazy_results.append(res)

results = dask.compute(*lazy_results, scheduler='distributed')


In [61]:
def calc_model_performance(model, X_test_tensor, dtest ):
    print('j')
    if nn.Sequential == type(model):
        pred = model(X_test_tensor)
        pred = helper.torch_tensor_to_numpy(pred)
        pred = pred.reshape(pred.shape[0],)
        
    elif xgb.core.Booster == type(model):
        pred = model.predict(dtest)

    return pred
    



In [63]:

comb_results = []

for comb_len in comb_lens:
    comb_models = combinations(best_models, comb_len)

    for i in list(comb_models):

        #print(i)
        # print(i[0])

        adj_r2_list = []
        comb_preds = []

        for model in i:

            if nn.Sequential == type(model):
                pred = model(X_test_tensor)
                pred = helper.torch_tensor_to_numpy(pred)
                pred = pred.reshape(pred.shape[0],)
                
            elif xgb.core.Booster == type(model):
                pred = model.predict(dtest)


            # pred_test = model.predict(dtest)
            comb_preds.append(pred)

        df_temp = pd.DataFrame(comb_preds)
        pred_mean = df_temp.mean()

        metirc_scores = []
        for metric in eval_metrics:
            metirc_score = metric(pred_mean, y_test)
            metirc_scores.append(metirc_score)

        # r2_mean = r2_score(pred_mean, y_test)
        # pred_mse = mean_squared_error(pred_mean, y_test)
        # mean_adj_r2 = np.mean(adj_r2_list)
        # comb_results.append([i, pred_mse, r2_mean])
        # print(metirc_scores)
        #metirc_scores = metirc_scores[0]
        comb_results.append([i, metirc_scores])
        # comb_results.extend([metirc_scores])
        #print()
        



KeyboardInterrupt: 

In [ ]:
from operator import itemgetter
best_adj_r2 = min(comb_results, key=itemgetter(1))
best_adj_r2_index = comb_results.index(best_adj_r2)
best_models = comb_results[1][best_adj_r2_index]

In [11]:
comb_lens = np.arange(1, len(best_models))
comb_lens

array([1, 2, 3, 4, 5, 6, 7, 8])

In [12]:

comb_results = []

In [44]:
temp = []

pred = nn_model(X_test_tensor)
pred = helper.torch_tensor_to_numpy(pred)

pred = pred.reshape(pred.shape[0],)
temp.append(pred)
temp.append(pred)

In [45]:
pred.reshape(pred.shape[0],)

array([ 833.4447  ,  278.66904 , 1630.7098  ,   56.1162  ,   51.830235,
         59.075565,   50.701183, 1032.8434  ,  980.1556  , 3420.1572  ,
        521.8256  ,  104.44682 ,  808.9032  ,  112.24868 ,  102.65138 ,
        108.791885, 1385.2935  , 1718.0125  , 1182.4812  , 1588.2902  ,
        767.52106 , 3210.3022  , 2083.584   , 1940.5717  ,  597.9002  ,
       2910.2317  , 2935.9883  , 2093.4048  ,   54.425945,   57.13496 ,
        102.69351 ,   56.842503,  872.63727 ,  431.98892 , 1684.185   ,
       3147.288   ,  591.23773 ,  364.73972 , 3223.7043  ,  784.0751  ,
       2034.9476  ,  283.62207 ,  770.0699  ,  267.5037  , 2226.8862  ,
       2146.7812  ,  823.43176 ,   57.449554, 1428.9481  ,  102.53839 ,
        101.95662 ,   50.82813 ,  600.75116 ,  263.68716 , 2450.3208  ,
        103.04074 ,   95.13579 , 1371.4115  , 1960.1312  ,  693.62286 ,
        502.6815  ,  745.9508  ,  847.6993  , 3169.7854  ], dtype=float32)

In [46]:
import pandas as pd
df_temp = pd.DataFrame(temp)

In [52]:
pred_mean = df_temp.mean()

In [54]:
mean_squared_error(pred, y_test)

1012302.3437202917

In [66]:
import pandas as pd

comb_results = []

for comb_len in comb_lens:
    comb_models = combinations(best_models, comb_len)

    for i in list(comb_models):
        adj_r2_list = []
        comb_preds = []

        for model in i:
            pred = model(X_test_tensor)
            pred = helper.torch_tensor_to_numpy(pred)
            pred = pred.reshape(pred.shape[0],)


#            r2_test = r2_score(helper.torch_tensor_to_numpy(pred), y_test)
#            adj_r2_test = helper.adjusted_r2(r2_test, X_test.shape[0], X_test.shape[1])
#            adj_r2_list.append(adj_r2_test)
            #print(adj_r2_test)
            comb_preds.append(pred)

        df_temp = pd.DataFrame(comb_preds)
        pred_mean = df_temp.mean()
        r2_mean = r2_score(pred_mean, y_test)
        pred_mse = mean_squared_error(pred_mean, y_test)
        # mean_adj_r2 = np.mean(adj_r2_list)
        comb_results.append([i, pred_mse, r2_mean])
        #print()



In [68]:
# comb_results

In [13]:

for comb_len in comb_lens:
    comb_models = combinations(best_models, comb_len)

    for i in list(comb_models):
        adj_r2_list = []
        for model in i:
            pred = model(X_test_tensor)
            r2_test = r2_score(helper.torch_tensor_to_numpy(pred), y_test)
            adj_r2_test = helper.adjusted_r2(r2_test, X_test.shape[0], X_test.shape[1])
            adj_r2_list.append(adj_r2_test)
            #print(adj_r2_test)
        mean_adj_r2 = np.mean(adj_r2_list)
        comb_results.append([i, mean_adj_r2])
        #print()



In [14]:
# comb_results

In [1]:
from operator import itemgetter
best_adj_r2 = min(comb_results, key=itemgetter(1))
best_adj_r2_index = comb_results.index(best_adj_r2)
best_models = comb_results[1][best_adj_r2_index]

NameError: name 'comb_results' is not defined

In [76]:
best_adj_r2_index, best_adj_r2

(127,
 [(Sequential(
     (0): Linear(in_features=40, out_features=49, bias=True)
     (1): ReLU()
     (2): Linear(in_features=49, out_features=112, bias=True)
     (3): ReLU()
     (4): Linear(in_features=112, out_features=251, bias=True)
     (5): ReLU()
     (6): Linear(in_features=251, out_features=206, bias=True)
     (7): ReLU()
     (8): Linear(in_features=206, out_features=232, bias=True)
     (9): ReLU()
     (10): Linear(in_features=232, out_features=73, bias=True)
     (11): ReLU()
     (12): Linear(in_features=73, out_features=1, bias=True)
     (13): ReLU()
   ),
   Sequential(
     (0): Linear(in_features=40, out_features=143, bias=True)
     (1): ReLU()
     (2): Linear(in_features=143, out_features=228, bias=True)
     (3): ReLU()
     (4): Linear(in_features=228, out_features=111, bias=True)
     (5): ReLU()
     (6): Linear(in_features=111, out_features=226, bias=True)
     (7): ReLU()
     (8): Linear(in_features=226, out_features=187, bias=True)
     (9): ReLU()
  

In [16]:
best_models

-2.186434306749301

In [17]:
best_adj_r2

[(Sequential(
    (0): Linear(in_features=40, out_features=110, bias=True)
    (1): ReLU()
    (2): Linear(in_features=110, out_features=159, bias=True)
    (3): ReLU()
    (4): Linear(in_features=159, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=189, bias=True)
    (7): ReLU()
    (8): Linear(in_features=189, out_features=1, bias=True)
    (9): ReLU()
  ),),
 -25.070263556190763]

In [76]:
# comb_models[best_adj_r2_index]


In [8]:
pred_dnn = nn_model(X_test_tensor)
mean_squared_error(helper.torch_tensor_to_numpy(pred_dnn), y_test)

1318375.6572819052

In [9]:
from sklearn.metrics import r2_score
r2_score(helper.torch_tensor_to_numpy(pred_dnn), y_test)

-1.385444947728498

In [8]:
X_test.shape

(64, 40)